In [10]:
import os
import shutil
import pandas as pd
from smartsheet_dataframe import get_as_df

In [11]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)
# Get only the WRR items
wrr_df = mss_df.loc[mss_df['UniqueID'].str.contains("WRR")].copy()

# Shuttle WRR In Process PDFs from "old" ticketid folder structure to "new" folder structure
## 24.11.06
## Steps:
1. Associate old TIDs to New TIDs
2. Locate folders/pdfs based on old TIDs
3. Transfer folders/pdfs to new TIDs
    - How to manage aggregation where it didn't exist before? (multiple week 1s for example)
4. Upload to G-drive

In [65]:
# Transfer function from the WAR project
# WAR didn't have any new aggregations of activities already in process.
# WRR does - so I expect to have issues of multiple week labels

def copy_pdfs_to_temp(src_directory, temp_directory):
    os.makedirs(temp_directory, exist_ok=True)
    
    for week_folder in os.listdir(src_directory):
        week_path = os.path.join(src_directory, week_folder)
        
        if os.path.isdir(week_path) and week_folder.startswith("Week"):
            new_week_path = os.path.join(temp_directory, week_folder)
            os.makedirs(new_week_path, exist_ok=True)
            
            for file_name in os.listdir(week_path):
                if file_name.lower().endswith("pdf"):
                    file_path = os.path.join(week_path, file_name)
                    shutil.copy(file_path, new_week_path)
                    print(f"Copied: {file_path} to {new_week_path}")

In [13]:
# Get lookup for Old Ticket ID to Unique ID
inprog_wrr_df = wrr_df[wrr_df['Status'].isin(['IN PROCESS', 'POST'])].copy()
inprog_wrr_oldTID = inprog_wrr_df.groupby(['TicketID'])['UniqueID'].apply(list).reset_index()
inprog_wrr_oldDict = dict(zip(inprog_wrr_oldTID['TicketID'], inprog_wrr_oldTID['UniqueID']))

In [14]:
# Get lookup for New Ticket ID from other notebook
new_tid_df = pd.read_excel(r"./new_tid.xlsx")

In [15]:
# Get the list of Unique IDs we need to care about from the old_tid dict
uids = []
for v in inprog_wrr_oldDict.values():
    if len(v) > 1:
        for id in v:
            uids.append(id)
    else:
        uids.append(v[0])
    
uids

['WRR230829CC03',
 'WRR230829CC02',
 'WRR230829CC01',
 'WRR230829TH01',
 'WRR230829TH02',
 'WRR231107CC01',
 'WRR231107CC02',
 'WRR231107CC03',
 'WRR231107TH01',
 'WRR231107TT01',
 'WRR240101TH01',
 'WRR240101TT01',
 'WRR240919TT01',
 'WRR240919CC06',
 'WRR240101CC03',
 'WRR240101CC02',
 'WRR240101CC01',
 'WRR240404TH01',
 'WRR240404TH02',
 'WRR240709CC03',
 'WRR240709CC02',
 'WRR240709CC01',
 'WRR240802TH01',
 'WRR240802TH03']

In [22]:
inprog_wrr_oldTID

,TicketID,UniqueID
0,WRR230829CCXX,"[WRR230829CC03, WRR230829CC02, WRR230829CC01]"
1,WRR230829TH01,[WRR230829TH01]
2,WRR230829TH02,[WRR230829TH02]
3,WRR231107CC01,[WRR231107CC01]
4,WRR231107CC02,[WRR231107CC02]
5,WRR231107CC03,[WRR231107CC03]
6,WRR231107TH01,[WRR231107TH01]
7,WRR231107TT01,[WRR231107TT01]
8,WRR240101TH01,[WRR240101TH01]
9,WRR240101TT01,[WRR240101TT01]


In [36]:
for id in uids:
    new_tid = new_tid_df.loc[new_tid_df['UniqueID'] == id, 'TicketID'].values[0]
    old_tid = wrr_df.loc[wrr_df['UniqueID'] == id, 'TicketID'].values[0]
    if old_tid != new_tid:
        print(id, old_tid, new_tid)

WRR231107CC02 WRR231107CC02 WRR231107XXXX
WRR231107TT01 WRR231107TT01 WRR231107XXXX


These two will be a bit of a pain, but if it comes down to it we can just do that by hand

### Let's Think

-I have a list of unique ids that have an active harvest management.
-I have a dictionary that ties this list to an old tid where the pdfs are stored
-I have a dataframe that ties this list to the new tid that will be the naming convention of the new location for the pdfs

Todo:
- Get list of old TID
- Get folder location of old TID
- Find associated UID
- Find new TID based on UID
- Create folder based on new TID
  - For our problem children maybe use the UniqueID to separate them?
    - What does that look like?
      - If New TID has 2 old TID associated then: uid folder names?
- Move over all pdfs

In [38]:
old_tid_list = []
for id in uids:
    old_tid = wrr_df.loc[wrr_df['UniqueID'] == id, 'TicketID'].values[0]
    old_tid_list.append(old_tid)

In [39]:
old_tid_list

['WRR230829CCXX',
 'WRR230829CCXX',
 'WRR230829CCXX',
 'WRR230829TH01',
 'WRR230829TH02',
 'WRR231107CC01',
 'WRR231107CC02',
 'WRR231107CC03',
 'WRR231107TH01',
 'WRR231107TT01',
 'WRR240101TH01',
 'WRR240101TT01',
 'WRR240101XXXX',
 'WRR240101XXXX',
 'WRR240101XXXX',
 'WRR240101XXXX',
 'WRR240101XXXX',
 'WRR240404TH01',
 'WRR240404TH02',
 'WRR240709CCXX',
 'WRR240709CCXX',
 'WRR240709CCXX',
 'WRR240802TH01',
 'WRR240802TH03']

In [68]:
for id in list(set(old_tid_list)):
    src_dir = fr"P:\Temp\Intern\MillTixDataEntry\{id}"
    uids = wrr_df.loc[wrr_df['TicketID'] == id, 'UniqueID'].values
    # Try and see if old ticketid has multiple new ticket ids
    new_ticketid = list(set(new_tid_df.loc[new_tid_df['UniqueID'].isin(uids), 'TicketID'].values))
    new_ticketid = new_ticketid[0]
    if new_ticketid == "WRR231107CC03":
        print(f"passing {new_ticketid}")
        continue
    print(f"moving from: {id} to: {new_ticketid}")
    temp_dir = fr"C:\Users\bbrown\OneDrive - Green Assets\Desktop\tmp\wrr\{new_ticketid}"

    copy_pdfs_to_temp(src_dir, temp_dir)

moving from: WRR231107CC01 to: WRR231107CC01
Copied: P:\Temp\Intern\MillTixDataEntry\WRR231107CC01\Week01\Mayhaw Settlement WE 12.23.2023.pdf to C:\Users\bbrown\OneDrive - Green Assets\Desktop\tmp\wrr\WRR231107CC01\Week01
Copied: P:\Temp\Intern\MillTixDataEntry\WRR231107CC01\Week02\1Mayhaw Settlement WE 12.30.2023 wk 21.pdf to C:\Users\bbrown\OneDrive - Green Assets\Desktop\tmp\wrr\WRR231107CC01\Week02
Copied: P:\Temp\Intern\MillTixDataEntry\WRR231107CC01\Week03\1W19 Mayhaw Settlement WE 12.16.2023.pdf to C:\Users\bbrown\OneDrive - Green Assets\Desktop\tmp\wrr\WRR231107CC01\Week03
Copied: P:\Temp\Intern\MillTixDataEntry\WRR231107CC01\Week03\W19 Mayhaw 121-1 Stand 5 WE 12.16.2023.pdf to C:\Users\bbrown\OneDrive - Green Assets\Desktop\tmp\wrr\WRR231107CC01\Week03
moving from: WRR231107TT01 to: WRR231107XXXX
Copied: P:\Temp\Intern\MillTixDataEntry\WRR231107TT01\Week01\Updated Mayhaw Settlement WE 06.29.24.pdf to C:\Users\bbrown\OneDrive - Green Assets\Desktop\tmp\wrr\WRR231107XXXX\Week01
